# Tidy Tuesday: What is on Netflix?
**July 29, 2025**

Today's goal is to analyze what's being viewed on Netflix so far in 2025.

## Imports

In [146]:
# Import libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [147]:
# Import data
movies = pd.read_csv('movies.csv')
shows = pd.read_csv('shows.csv')

## Clean the data
**Cleaning the `movies` data first**

In [148]:
# Preview data
movies.head()

,source,report,title,available_globally,release_date,hours_viewed,runtime,views
0,1_What_We_Watched_A_Netflix_Engagement_Report_...,2025Jan-Jun,Back in Action,Yes,2025-01-17,313000000.0,1H 54M 0S,164700000.0
1,1_What_We_Watched_A_Netflix_Engagement_Report_...,2025Jan-Jun,STRAW,Yes,2025-06-06,185200000.0,1H 48M 0S,102900000.0
2,1_What_We_Watched_A_Netflix_Engagement_Report_...,2025Jan-Jun,The Life List,Yes,2025-03-28,198900000.0,2H 5M 0S,95500000.0
3,1_What_We_Watched_A_Netflix_Engagement_Report_...,2025Jan-Jun,Exterritorial,Yes,2025-04-30,159000000.0,1H 49M 0S,87500000.0
4,1_What_We_Watched_A_Netflix_Engagement_Report_...,2025Jan-Jun,Havoc,Yes,2025-04-25,154900000.0,1H 47M 0S,86900000.0


`source` can be dropped. Irrelevant data.

In [149]:
# Drop source column
movies = movies.drop(columns='source')

# We want to only look at what's being viewd in 2025
movies.report.value_counts()

report
2023Jul-Dec    9399
2024Jan-Jun    9364
2024Jul-Dec    8684
2025Jan-Jun    8674
Name: count, dtype: int64

**8,674** movies have been recorded for 2025. I will create a mask below:

In [150]:
# Create mask
movies = movies[movies['report'] == '2025Jan-Jun']
movies.head()

,report,title,available_globally,release_date,hours_viewed,runtime,views
0,2025Jan-Jun,Back in Action,Yes,2025-01-17,313000000.0,1H 54M 0S,164700000.0
1,2025Jan-Jun,STRAW,Yes,2025-06-06,185200000.0,1H 48M 0S,102900000.0
2,2025Jan-Jun,The Life List,Yes,2025-03-28,198900000.0,2H 5M 0S,95500000.0
3,2025Jan-Jun,Exterritorial,Yes,2025-04-30,159000000.0,1H 49M 0S,87500000.0
4,2025Jan-Jun,Havoc,Yes,2025-04-25,154900000.0,1H 47M 0S,86900000.0


In [151]:
# Check data types
movies.dtypes

report                 object
title                  object
available_globally     object
release_date           object
hours_viewed          float64
runtime                object
views                 float64
dtype: object

I will like to store the `runtime` in the form of hours so I can compare calculations with the `hours_viewed` column. In addition, `release_date` could benefit from converting it into a *datetime* format.\
Otherwise, the data types are fine as it.

In [152]:
# Convert runtime to hours
movies = movies.dropna().reset_index(drop=True) # Drop NAs
# Split runtime up into hour, minute, sec
movies[['hour', 'min', 'sec']] = movies['runtime'].str.split(expand=True)
movies.head()

,report,title,available_globally,release_date,hours_viewed,runtime,views,hour,min,sec
0,2025Jan-Jun,Back in Action,Yes,2025-01-17,313000000.0,1H 54M 0S,164700000.0,1H,54M,0S
1,2025Jan-Jun,STRAW,Yes,2025-06-06,185200000.0,1H 48M 0S,102900000.0,1H,48M,0S
2,2025Jan-Jun,The Life List,Yes,2025-03-28,198900000.0,2H 5M 0S,95500000.0,2H,5M,0S
3,2025Jan-Jun,Exterritorial,Yes,2025-04-30,159000000.0,1H 49M 0S,87500000.0,1H,49M,0S
4,2025Jan-Jun,Havoc,Yes,2025-04-25,154900000.0,1H 47M 0S,86900000.0,1H,47M,0S


In [153]:
# Remove the non-numerical characters from the split up time columns
# Also convert into numericals
movies['hour'] = pd.to_numeric(movies['hour'].str[:-1])
movies['min'] = pd.to_numeric(movies['min'].str[:-1])
movies['sec'] = pd.to_numeric(movies['sec'].str[:-1])
movies.head()

,report,title,available_globally,release_date,hours_viewed,runtime,views,hour,min,sec
0,2025Jan-Jun,Back in Action,Yes,2025-01-17,313000000.0,1H 54M 0S,164700000.0,1,54,0.0
1,2025Jan-Jun,STRAW,Yes,2025-06-06,185200000.0,1H 48M 0S,102900000.0,1,48,0.0
2,2025Jan-Jun,The Life List,Yes,2025-03-28,198900000.0,2H 5M 0S,95500000.0,2,5,0.0
3,2025Jan-Jun,Exterritorial,Yes,2025-04-30,159000000.0,1H 49M 0S,87500000.0,1,49,0.0
4,2025Jan-Jun,Havoc,Yes,2025-04-25,154900000.0,1H 47M 0S,86900000.0,1,47,0.0


In [168]:
# Create column of movie length in hours
movie_length = []

for row in range(len(movies)):
    # Convert the hour, min, and sec values to hour
    time_hour = movies['hour'][row]
    time_min = movies['min'][row] / 60
    time_sec = movies['sec'][row] / 60 / 60
    # Add the time values up
    movie_length.append( round(time_hour + time_min + time_sec, 2) )

# New column introduced
movies['movie_length'] = movie_length
movies.head()

,report,title,available_globally,release_date,hours_viewed,runtime,views,hour,min,sec,movie_length
0,2025Jan-Jun,Back in Action,Yes,2025-01-17,313000000.0,1H 54M 0S,164700000.0,1,54,0.0,1.90
1,2025Jan-Jun,STRAW,Yes,2025-06-06,185200000.0,1H 48M 0S,102900000.0,1,48,0.0,1.80
2,2025Jan-Jun,The Life List,Yes,2025-03-28,198900000.0,2H 5M 0S,95500000.0,2,5,0.0,2.08
3,2025Jan-Jun,Exterritorial,Yes,2025-04-30,159000000.0,1H 49M 0S,87500000.0,1,49,0.0,1.82
4,2025Jan-Jun,Havoc,Yes,2025-04-25,154900000.0,1H 47M 0S,86900000.0,1,47,0.0,1.78


In [179]:
# Convert release_date to datetime
movies['release_date'] = pd.to_datetime(movies['release_date'])

# Drop report, runtime, and the time splits
movies = movies.drop(columns=['report', 'runtime', 'hour', 'min', 'sec'])
movies.head()

,title,available_globally,release_date,hours_viewed,views,movie_length
0,Back in Action,Yes,2025-01-17,313000000.0,164700000.0,1.90
1,STRAW,Yes,2025-06-06,185200000.0,102900000.0,1.80
2,The Life List,Yes,2025-03-28,198900000.0,95500000.0,2.08
3,Exterritorial,Yes,2025-04-30,159000000.0,87500000.0,1.82
4,Havoc,Yes,2025-04-25,154900000.0,86900000.0,1.78


In [181]:
# Final check for data types
movies.dtypes

title                         object
available_globally            object
release_date          datetime64[ns]
hours_viewed                 float64
views                        float64
movie_length                 float64
dtype: object

All set for movies.

## Movie analysis
Some questions to answer:
* What are the top 10 popular movies being watched in 2025?
* What are the popular movies released in 2025?
* What are the longest movies being most watched? The shortest?
* Does global availability matter?